In [18]:
import pathlib
# BASE_DIR will be like '/home/jovyan/DemoExample/'
BASE_DIR = pathlib.Path().absolute()
print(f"Working dir: {BASE_DIR}")

Working dir: /notebook/greenAI


In [19]:
# !pip install transformers datasets

In [20]:
import torch
from torch.nn import functional as F
# from transformers import BertModel, BertConfig, BertTokenizer, BertLMHeadModel, BertTokenizerFast
from transformers import *
from datasets import load_dataset

import os
import sys
from tqdm import tqdm
import random
import math

In [21]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

'cuda'

In [25]:
!pwd

/notebook/greenAI


In [10]:
def init_model(model_type):
    if model_type == 'bert':
        tokenizer = BertTokenizerFast.from_pretrained("bert-base-uncased")
        model = BertLMHeadModel.from_pretrained('bert-base-uncased',
                                                return_dict=True, 
                                                is_decoder=True)
    elif model_type == 'scibert':
        tokenizer = BertTokenizerFast.from_pretrained("allenai/scibert_scivocab_uncased")
        model = BertLMHeadModel.from_pretrained('allenai/scibert_scivocab_uncased',
                                                return_dict=True, 
                                                is_decoder=True)
        # tokenizer = AutoTokenizer.from_pretrained('allenai/scibert_scivocab_uncased')
        # model = AutoModelWithLMHead.from_pretrained('allenai/scibert_scivocab_uncased')
    elif model_type == 'gpt2':
        tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
        model = GPT2LMHeadModel.from_pretrained("gpt2",
                                                return_dict=True,
                                                is_decoder=True)
    else:
        raise NotImplementedError(model_type)
        
    return tokenizer, model

tokenizer, model = init_model('gpt2')
model = model.to(device)

loading file https://huggingface.co/gpt2/resolve/main/vocab.json from cache at /root/.cache/huggingface/transformers/684fe667923972fb57f6b4dcb61a3c92763ad89882f3da5da9866baf14f2d60f.c7ed1f96aac49e745788faa77ba0a26a392643a50bb388b9c04ff469e555241f
loading file https://huggingface.co/gpt2/resolve/main/merges.txt from cache at /root/.cache/huggingface/transformers/c0c761a63004025aeadd530c4c27b860ec4ecbe8a00531233de21d865a402598.5d12962c5ee615a4c803841266e9c3be9a691a924f72d395d3a6c6c81157788b
loading file https://huggingface.co/gpt2/resolve/main/added_tokens.json from cache at None
loading file https://huggingface.co/gpt2/resolve/main/special_tokens_map.json from cache at None
loading file https://huggingface.co/gpt2/resolve/main/tokenizer_config.json from cache at None
loading configuration file https://huggingface.co/gpt2/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/fc674cd6907b4c9e933cb42d67662436b89fa9540a1f40d7c919d0109289ad01.7d2e0efa5ca20cef4fb1993821

In [15]:
tokenizer, model.config

(PreTrainedTokenizer(name_or_path='gpt2', vocab_size=50257, model_max_len=1024, is_fast=False, padding_side='right', truncation_side='right', special_tokens={'bos_token': AddedToken("<|endoftext|>", rstrip=False, lstrip=False, single_word=False, normalized=True), 'eos_token': AddedToken("<|endoftext|>", rstrip=False, lstrip=False, single_word=False, normalized=True), 'unk_token': AddedToken("<|endoftext|>", rstrip=False, lstrip=False, single_word=False, normalized=True)}),
 GPT2Config {
   "_name_or_path": "gpt2",
   "activation_function": "gelu_new",
   "architectures": [
     "GPT2LMHeadModel"
   ],
   "attn_pdrop": 0.1,
   "bos_token_id": 50256,
   "embd_pdrop": 0.1,
   "eos_token_id": 50256,
   "initializer_range": 0.02,
   "is_decoder": true,
   "layer_norm_epsilon": 1e-05,
   "model_type": "gpt2",
   "n_ctx": 1024,
   "n_embd": 768,
   "n_head": 12,
   "n_inner": null,
   "n_layer": 12,
   "n_positions": 1024,
   "reorder_and_upcast_attn": false,
   "resid_pdrop": 0.1,
   "scale_

In [16]:
# inputs = tokenizer("What do you plan to do with ", return_tensors="pt")
# outputs = model(**inputs)

# logits = outputs.logits
# logits.shape, inputs['input_ids'].shape, \
# logits.argmax(dim=-1), inputs['input_ids'], \
# tokenizer.decode(logits.argmax(dim=-1)[0])

In [17]:
def getListOfFiles(dirName):
    # create a list of file and sub directories 
    # names in the given directory 
    listOfFile = os.listdir(dirName)
    allFiles = list()
    # Iterate over all the entries
    for entry in listOfFile:
        # Create full path
        fullPath = os.path.join(dirName, entry)
        # If entry is a directory then get the list of files in this directory 
        if os.path.isdir(fullPath):
            allFiles = allFiles + getListOfFiles(fullPath)
        else:
            allFiles.append(fullPath)
                
    return allFiles

filelist = getListOfFiles("../data/sci-hub/txt")
len(filelist)

FileNotFoundError: [Errno 2] No such file or directory: '../data/sci-hub/txt'

In [46]:
!nvidia-smi

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
Tue Apr 19 01:15:19 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 418.165.02   Driver Version: 418.165.02   CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM3...  On   | 00000000:B9:00.0 Off |                    0 |
| N/A   32C    P0    66W / 350W |   3760MiB / 32480MiB |      0%      Default |
+---

In [14]:
# bert
sizes = []
token_length = []
k = 100
s = 0
l = 0 
random.seed(0)
for file in random.sample(filelist, k):
    f = open(file, "r")
    encodings = tokenizer(f.read(), return_tensors="pt")
    f.close()
    token_length.append(encodings['input_ids'].shape[1])
    sizes.append(os.path.getsize(file) / 1024)
mean_size = sum(sizes) / k
std_size = math.sqrt(sum((s - mean_size)**2 for s in sizes) / k)
mean_length = sum(token_length) / k 
std_length = math.sqrt(sum((l - mean_length)**2 for l in token_length) / k)
mean_size, std_size, mean_length, std_length

NameError: name 'filelist' is not defined

In [63]:
# scibert
sizes = []
token_length = []
k = 100
s = 0
l = 0 
random.seed(0)
for file in random.sample(filelist, k):
    f = open(file, "r")
    encodings = tokenizer(f.read(), return_tensors="pt")
    f.close()
    token_length.append(encodings['input_ids'].shape[1])
    sizes.append(os.path.getsize(file) / 1024)
mean_size = sum(sizes) / k
std_size = math.sqrt(sum((s - mean_size)**2 for s in sizes) / k)
mean_length = sum(token_length) / k 
std_length = math.sqrt(sum((l - mean_length)**2 for l in token_length) / k)
mean_size, std_size, mean_length, std_length

(27.10625, 16.21626417098988, 6837.57, 4281.092078558928)

In [60]:
# gpt2
sizes = []
token_length = []
k = 100
s = 0
l = 0 
random.seed(0)
for file in random.sample(filelist, k):
    f = open(file, "r")
    encodings = tokenizer(f.read(), return_tensors="pt")
    f.close()
    token_length.append(encodings['input_ids'].shape[1])
    sizes.append(os.path.getsize(file) / 1024)
mean_size = sum(sizes) / k
std_size = math.sqrt(sum((s - mean_size)**2 for s in sizes) / k)
mean_length = sum(token_length) / k 
std_length = math.sqrt(sum((l - mean_length)**2 for l in token_length) / k)
mean_size, std_size, mean_length, std_length

Token indices sequence length is longer than the specified maximum sequence length for this model (2255 > 1024). Running this sequence through the model will result in indexing errors


(27.10625, 16.21626417098988, 8276.01, 5341.004460763911)

In [30]:
# random.seed(1)
# k = 100
# samplelist = random.sample(filelist, k)
# dataset = load_dataset('text', data_files={'test': samplelist})

In [31]:
# encodings = tokenizer("\n\n".join(dataset['test']["text"]), return_tensors="pt")
# encodings['input_ids'].shape

In [24]:
# !nvidia-smi

In [223]:
max_length = model.config.max_position_embeddings
stride = max_length // 2
print(max_length, stride)
ppls = []
for seed in range(10):

    random.seed(seed)
    k = 10
    samplelist = random.sample(filelist, k)
    dataset = load_dataset('text', data_files={'test': samplelist})
    encodings = tokenizer("\n\n".join(dataset['test']["text"]), return_tensors="pt")

    nlls = []
    for i in tqdm(range(0, encodings.input_ids.size(1), stride)):
        begin_loc = max(i + stride - max_length, 0)
        end_loc = min(i + stride, encodings.input_ids.size(1))
        trg_len = end_loc - i  # may be different from stride on last loop
        input_ids = encodings.input_ids[:, begin_loc:end_loc].to(device)
        target_ids = input_ids.clone()
        target_ids[:, :-trg_len] = -100
        # print(begin_loc, end_loc, trg_len)
        # print(input_ids, target_ids)

        with torch.no_grad():
            outputs = model(input_ids, labels=target_ids)
            # outputs = model(input_ids, target_ids)
            neg_log_likelihood = outputs[0] * trg_len

        nlls.append(neg_log_likelihood)

    ppl = torch.exp(torch.stack(nlls).sum() / end_loc)
    ppls.append(ppl.item())

1024 512


Using custom data configuration default-6c5203eb06a1fecd
Reusing dataset text (/home/jovyan/.cache/huggingface/datasets/text/default-6c5203eb06a1fecd/0.0.0/4b86d314f7236db91f0a0f5cda32d4375445e64c5eda2692655dd99c2dac68e8)


  0%|          | 0/1 [00:00<?, ?it/s]

Token indices sequence length is longer than the specified maximum sequence length for this model (70853 > 1024). Running this sequence through the model will result in indexing errors
100%|██████████| 139/139 [00:04<00:00, 29.78it/s]
Using custom data configuration default-55c08f045c4f62f3
Reusing dataset text (/home/jovyan/.cache/huggingface/datasets/text/default-55c08f045c4f62f3/0.0.0/4b86d314f7236db91f0a0f5cda32d4375445e64c5eda2692655dd99c2dac68e8)


  0%|          | 0/1 [00:00<?, ?it/s]

...

100%|██████████| 266/266 [00:09<00:00, 29.50it/s]
Using custom data configuration default-58e3c1df4baab94b
Reusing dataset text (/home/jovyan/.cache/huggingface/datasets/text/default-58e3c1df4baab94b/0.0.0/4b86d314f7236db91f0a0f5cda32d4375445e64c5eda2692655dd99c2dac68e8)


  0%|          | 0/1 [00:00<?, ?it/s]

100%|██████████| 133/133 [00:04<00:00, 29.59it/s]
Using custom data configuration default-8f72c2ea9e9e5631
Reusing dataset text (/home/jovyan/.cache/huggingface/datasets/text/default-8f72c2ea9e9e5631/0.0.0/4b86d314f7236db91f0a0f5cda32d4375445e64c5eda2692655dd99c2dac68e8)


  0%|          | 0/1 [00:00<?, ?it/s]

100%|██████████| 150/150 [00:05<00:00, 29.55it/s]


In [224]:
text = 'knowledge concerning the problem is limited, but much important information and research is available in the fields of education, neurology, psychology, psychiatry, and ophthalmology. with any unsolved problem there is much disagreement on terminology. while this adds to the initial confusion, i t accentuates the need for physicians and educators t o coordinate their efforts. the phys. stat. sol. ( a ) 194, no. 2, 460 – 463 ( 2002 ) large area algan / gan hemts grown on insulating silicon carbide substrates r. lossy1 ) ( a ), n. chaturvedi ( a ), p. heymann ( a ), j. wurfl ( a ), s. muller ( b ) and k. kohler ( b ) ( a ) ferdinand - braun - institut fur hochstfrequenztechnik, albert - einstein - straße 11, 12489 berlin, germany ( b ) fraunhofer - institut fur angewandte festkorperphysik, tullastraße 72, 79108 freiburg, germany ( received july 22, 2002 ; accepted october 1, 2002 ) pacs : 73. 40. kp ; 85. 30. de large periphery al0. 25ga0. 75n / gan - hemts on sic - substrates are fabricated on a 2 - inch process line using stepper lithography. dc characteristics reveal current densities above 1. 2 a / mm and intrinsic transconductances of 360 ms / mm. depending on device size the maximum frequency of oscillation fmax varies from 27 – 79 ghz. with these devices a power density of 5. 2 w / mm and a power level of 13. 8 w is achieved at 2 ghz. introduction due to their high breakdown field and excellent electron transport properties wide - bandgap algan / gan high electron mobility transistors ( hemts ) offer several advantages for high power microwave applications compared to competitive technologies. these advantages include high operation voltage, high impedance for power matching conditions and high cut - off frequency. furthermore, sic as a substrate material provides an excellent thermal conductivity, comparable to copper which is mandatory for effective heat removal in high power'

In [22]:
input_ids = tokenizer.encode(text, return_tensors="pt").to(device)
tokenizer.decode(model(input_ids).logits.argmax(dim=-1)[0])

NameError: name 'text' is not defined

In [226]:
# gpt2
ppls, sum(ppls) / len(ppls)

([24.465299606323242,
  19.73772430419922,
  21.987688064575195,
  18.808181762695312,
  19.504535675048828,
  22.657756805419922,
  19.79397964477539,
  24.842866897583008,
  22.646448135375977,
  21.318130493164062],
 21.576261138916017)

In [217]:
# scibert
ppls, sum(ppls) / len(ppls)

([6694142.5,
  9345072.0,
  8455978.0,
  8464676.0,
  9313896.0,
  9941689.0,
  9422571.0,
  13062048.0,
  11012341.0,
  7944231.5],
 9365664.5)

In [213]:
# bert
ppls, sum(ppls) / len(ppls)

([13253.4609375,
  10427.14453125,
  17386.408203125,
  16279.7548828125,
  12566.482421875,
  13471.0205078125,
  10982.76953125,
  13014.9013671875,
  13869.439453125,
  12528.5498046875],
 13377.9931640625)

In [23]:
max_length = model.config.max_position_embeddings
stride = max_length // 2
print(max_length, stride)

test = load_dataset("wikitext", "wikitext-2-raw-v1", split="test")
encodings = tokenizer("\n\n".join(test["text"]), return_tensors="pt")

nlls = []
for i in tqdm(range(0, encodings.input_ids.size(1), stride)):
    begin_loc = max(i + stride - max_length, 0)
    end_loc = min(i + stride, encodings.input_ids.size(1))
    trg_len = end_loc - i  # may be different from stride on last loop
    input_ids = encodings.input_ids[:, begin_loc:end_loc].to(device)
    target_ids = input_ids.clone()
    target_ids[:, :-trg_len] = -100
    # print(begin_loc, end_loc, trg_len)
    # print(input_ids, target_ids)

    with torch.no_grad():
        outputs = model(input_ids, labels=target_ids)
        # outputs = model(input_ids, target_ids)
        neg_log_likelihood = outputs[0] * trg_len

    nlls.append(neg_log_likelihood)

ppl = torch.exp(torch.stack(nlls).sum() / end_loc)

1024 512


Reusing dataset wikitext (/root/.cache/huggingface/datasets/wikitext/wikitext-2-raw-v1/1.0.0/a241db52902eaf2c6aa732210bead40c090019a499ceb13bcbfa3f8ab646a126)
Token indices sequence length is longer than the specified maximum sequence length for this model (287644 > 1024). Running this sequence through the model will result in indexing errors
100%|██████████| 562/562 [00:57<00:00,  9.78it/s]


In [24]:
# gpt 2
ppl

tensor(25.1738, device='cuda:0')

In [235]:
# bert-base
ppl

tensor(8649.4365, device='cuda:0')

In [240]:
# scibert
ppl

tensor(8803313., device='cuda:0')